# What this notebook is

👨‍💻: This is our best attempt so far in the day to get a model shipped to prod. It corresponds to the Iteration 5 of our "EDA" notebook.

<hr/>
<p style="font-size: 1.3em; color: red;">START: Dependencies</p>

<p style="font-size: 1.3em; color: red;">End of section: Dependencies. STOP HERE AND GO BACK TO INSTRUCTIONS</p>
<hr/>

<hr/>
<p style="font-size: 1.3em; color: red;">START: Training a Boosting Classifier with clustering (iteration 5)</p>

In [ ]:
from sklearn.cluster import KMeans, Birch, MiniBatchKMeans

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

👨‍💻: Fully specified path, TODO: Check where

In [ ]:
df = pd.read_csv('/home/emilio/Repos/work/ivado/ml-powered-bank-marketing-solution/before_everything_there_were_notebooks/data/csv/bank_marketing_2008-05-01_to_2010-07-31.csv', sep=';')

# This is necessary because we had to rename the columns at some point...
df = df.rename(columns={"comm_type": "contact",
"comm_month": "month",
"comm_day": "day_of_week",
"comm_duration": "duration",
"curr_n_contact": "campaign",
"days_since_last_campaign": "pdays",
"last_n_contact": "previous",
"last_outcome": "poutcome",
"curr_outcome": "y"})

In [ ]:
y = df.y.replace({'yes':1, 'no':0 })

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df.drop(columns='y'), y, train_size=0.8)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.8)

In [ ]:
train_x = train_x.drop(columns='duration')
val_x = val_x.drop(columns='duration')
test_x = test_x.drop(columns='duration')

# Training

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import RocCurveDisplay, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier,VotingClassifier
# classifier5 = VotingClassifier(estimators=[('a', DecisionTreeClassifier()), ('b', GradientBoostingClassifier())])
classifier5 = GradientBoostingClassifier()

# This is added after going on the notebook once:
from sklearn.base import BaseEstimator, TransformerMixin
class HasBeenCalledBeforeTransformer(BaseEstimator,TransformerMixin):
    def fit(self, X=None, y=None):
        return self
    def transform(self, X, y=None):
        X['has_been_called_before'] = (X.pdays != 999).astype(int)
        return X

In [ ]:
from sklearn.cluster import KMeans, Birch, MiniBatchKMeans

In [ ]:
person_info_cols_cat = ['job','marital','education','default', 'housing','loan', 'contact']
person_info_cols_num = ['age']

In [ ]:
kmeans_transformer = ColumnTransformer([
    ('scaleAge', StandardScaler(), person_info_cols_num),
    ('onehot', OneHotEncoder(sparse_output=True), person_info_cols_cat),
], remainder='drop')
kmeans_pipeline = Pipeline([
    ('c', kmeans_transformer),
    # ('cluster', Birch()),
    ('cluster', KMeans(n_clusters=12)),
])

In [ ]:
num_cols_wo_customer = ['campaign','pdays','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']
cat_cols_wo_customer = ['month','day_of_week', 'poutcome']

**NOTE THIS IS COPY PASTED FROM THE EDA, AND THERE IS A SUBTLE BUG: WE ARE USING ANOTHER TRANSFORMER (BUT SINCE WE ARE IN A NOTEBOOK THE INSTANCE EXISTED SO THAT MEANS OUR PREVIOUS EXPERIMENTS MIGHT BE WRONG.**

In [ ]:
transformerCategorical5 = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='error')),
])
transformerNumerical5 = Pipeline([
    # ('add_has_been_called_before', HasBeenCalledBeforeTransformer()),
    ('scale', StandardScaler()),
])
transformer5 = ColumnTransformer([
    ('cluster_customer', kmeans_pipeline, person_info_cols_cat + person_info_cols_num),
    ('num', transformerNumerical5, num_cols_wo_customer),
    ('cat',  transformerCategorical5, cat_cols_wo_customer),
])

pipeline_5 = Pipeline([
    ('data', transformer5),
    ('clf', classifier5),
])

<p style="font-size: 1.3em; color: red;">End of section: Training a Boosting Classifier with clustering (iteration 5) STOP HERE AND GO BACK TO INSTRUCTIONS</p>
<hr/>

<hr/>
<p style="font-size: 1.3em; color: red;">START: Continuing our iteration 5 after the bug is fixed</p>

In [ ]:
%%time
pipeline_5.fit(train_x, train_y)

In [ ]:
accuracy_score(val_y.values, pipeline_5.predict(val_x))

In [ ]:
RocCurveDisplay.from_predictions(test_y.values,pipeline_5.decision_function(test_x))

👨‍💻: Those are pretty good numbers! We are more or less the same thing as our iteration 5. Although is the AUC the same? Shouldn't it be exactly the same? 

👨‍💻: Oh well, to do later... For now, let's make sure we have a saved model

In [ ]:
# Let's save our model
import pickle
with open('best_model5.pkl', 'wb') as f:
    pickle.dump(pipeline_5, f)

<p style="font-size: 1.3em; color: red;">End of section: Continuing our iteration 5 after the bug is fixed. STOP HERE AND GO BACK TO INSTRUCTIONS</p>
<hr/>

<hr/>
<p style="font-size: 1.3em; color: red;">START: Attempting Cross Validation and something else?</p>

## Trying to do some cross validation to have a better model?

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
pd.concat([train_x, val_x])

In [ ]:
skf = StratifiedKFold(n_splits=3)
cv_x, cv_y = pd.concat([train_x, val_x]), pd.concat([train_y, val_y])
# for train_ix, test_ix in skf.split(cv_x, cv_y):
#     display(cv_x.iloc[train_ix])

In [ ]:
kmeans_transformerCV = ColumnTransformer([
    ('scaleAge', StandardScaler(), person_info_cols_num),
    ('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=True), person_info_cols_cat),
], remainder='drop')
kmeans_pipelineCV = Pipeline([
    ('c', kmeans_transformerCV),
    # ('cluster', Birch()),
    ('cluster', KMeans(n_clusters=12)),
])
classifierCV = GradientBoostingClassifier()

transformerCategoricalCV = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='error')),
])
transformerNumericalCV = Pipeline([
    # ('add_has_been_called_before', HasBeenCalledBeforeTransformer()),
    ('scale', StandardScaler()),
])
transformerCV = ColumnTransformer([
    ('cluster_customer', kmeans_pipelineCV, person_info_cols_cat + person_info_cols_num),
    ('num', transformerNumericalCV, num_cols_wo_customer),
    ('cat',  transformerCategoricalCV, cat_cols_wo_customer),
])

pipeline_CV = Pipeline([
    ('data', transformerCV),
    ('clf', classifierCV),
])

In [ ]:
np.positive(np.geomspace(2, 200, num=9, endpoint=False, dtype=np.int8))

👨‍💻: This package seems pretty cool, I've seen Intel give a good talk about it, it's about time we try it.

In [ ]:
! pip install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

🧑‍🔬: These are highly scientific values 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameter_grid = {
    'clf__max_depth': (2, 3, 4, 5),
    'clf__n_estimators': np.positive(np.geomspace(2, 200, num=9, endpoint=False, dtype=np.int8)),
    'clf__criterion': ('friedman_mse','squared_error'),
    'data__cluster_customer__cluster__n_clusters': (5,10,15,20,30)
}

random_search = RandomizedSearchCV(
    estimator=pipeline_CV,
    param_distributions=parameter_grid,
    n_iter=10,
    random_state=0,
    n_jobs=11,
    verbose=1,
)

In [ ]:
%%time
random_search.fit(cv_x, cv_y)

👨‍💻: That took a long time, although my company laptop is a 🥔

In [ ]:
random_search.best_estimator_

In [ ]:
# Note that we need to use test since we saw val
display(accuracy_score(test_y.values, pipeline_5.predict(test_x)))
display(RocCurveDisplay.from_predictions(test_y.values,pipeline_5.decision_function(test_x)))

In [ ]:
# Note that we need to use test since we saw val
display(accuracy_score(test_y.values, random_search.best_estimator_.predict(test_x)))
display(RocCurveDisplay.from_predictions(test_y.values,random_search.best_estimator_.decision_function(test_x)))

In [ ]:
pipeline_CV.get_params()

👨‍💻: Man these `params` name are going to kill me, they're soooooooo long

<p style="font-size: 1.3em; color: red;">End of section: Dependencies. STOP HERE AND GO BACK TO INSTRUCTIONS</p>
<hr/>

👨‍💻: At least the performance is _pretty nice_, I'm happy.

<hr/>
<p style="font-size: 1.3em; color: red;">START: XGBoost has entered the chat</p>

# Big boy league

👨‍💻: The scikit learn implementation was cool and all, but it was starting to become tedious to manipulate all objects. Let's see if we prefer `XGBoost`

## XGBoost

In [ ]:
! pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(transformer5.transform(train_x), label=train_y)
dtest = xgb.DMatrix(transformer5.transform(test_x), label=test_y)

In [ ]:
evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [ ]:
num_round = 30
param = {'max_depth': 4, 'eta': 0.6, 'objective': 'binary:logistic'}
param['nthread'] = 8
param['eval_metric'] = 'auc'
param['sampling_method'] = 'uniform'
param['gamma'] = 0.02
bst = xgb.train(param, dtrain, num_round, evals=evallist,early_stopping_rounds=20)

In [ ]:
display(accuracy_score(test_y.values, bst.predict(xgb.DMatrix(transformer5.transform(test_x)))))

In [ ]:
display(RocCurveDisplay.from_predictions(test_y.values,bst.predict(xgb.DMatrix(transformer5.transform(test_x)))))

In [ ]:
from xgboost import plot_importance

In [ ]:
ix_to_col = [ v for i,v in enumerate(transformer5.get_feature_names_out()) ]

In [ ]:
bst.feature_names = ix_to_col

In [ ]:
plot_importance(bst)

In [ ]:
ix_to_col[18]

## XGBoost without the kmeans

👨‍💻: Alright let's make things a bit cleaner and define some functions

In [ ]:
def create_prediction_pipeline(data_processor, classifier ) -> Pipeline:
    """Creates a prediction pipeline out of existing data processor and classifier"""
    pipeline = Pipeline([
        ('data_processing', data_processor),
        ('classifier', classifier)
    ])
    return pipeline

# Method to create a generic pipeline
def make_pipe_with_classifier(clf):
    numerical_pipeline = Pipeline([
        ('scaler', StandardScaler()),
    ])
    
    categorical_pipeline = Pipeline(steps=[
        # TODO: raise when there are unknown values
        ('one_hot_encoding', OneHotEncoder(handle_unknown='infrequent_if_exist')),
    ])
    transformer = ColumnTransformer([
        ('numerical_simple_scaler', numerical_pipeline, ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']),
        ('categorical_handler', categorical_pipeline, ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome'])
    ])
    pipeline = create_prediction_pipeline(transformer,clf)
    return pipeline

In [ ]:
simple_data_processing_pipeline = make_pipe_with_classifier(None).named_steps.data_processing

In [ ]:
scratch_dtrain = xgb.DMatrix(simple_data_processing_pipeline.fit_transform(train_x), label=train_y)
scratcg_dtest = xgb.DMatrix(simple_data_processing_pipeline.transform(val_x), label=val_y)

In [ ]:
%%time
scratch_param = {'max_depth': 3, 'eta': 0.6, 'objective': 'binary:logistic'}
scratch_param['nthread'] = 8
scratch_param['eval_metric'] = 'auc'
scratch_param['sampling_method'] = 'gradient_based'
scratch_param['gamma'] = 0.02
scratch_param['gpu_id'] = 0
scratch_param['tree_method'] = 'gpu_hist'
scratch_bst = xgb.train(scratch_param, scratch_dtrain, 50, evals=[(scratch_dtrain, 'train'), (scratcg_dtest, 'eval')],early_stopping_rounds=20)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

In [ ]:
display(RocCurveDisplay.from_predictions(test_y.values,scratch_bst.predict(xgb.DMatrix(simple_data_processing_pipeline.transform(test_x)))))

👨‍💻: Don't ask me how I did this, I _don't_ remember. Actually I don't remember coding it, must have been a vision...

In [ ]:
scratch_bst.feature_names = [ v for i,v in enumerate(simple_data_processing_pipeline.get_feature_names_out()) ]
plot_importance(scratch_bst, max_num_features=20)

**So it seems xgboost is _pretty_ fast and could make the job easier that will be our real model**

In [ ]:
# Let's save it
import pickle
scratch_bst.save_model('best_xgb_clf.model')
with open('best_xgb_data_processing.pkl', 'wb') as f:
    pickle.dump(simple_data_processing_pipeline, f)

<p style="font-size: 1.3em; color: red;">End of section: XGBoost has entered the chat STOP HERE AND GO BACK TO INSTRUCTIONS</p>
<hr/>